### Create option_data.csv file

In [2]:
#Import the necessary packages 
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib as mpl  
%matplotlib inline
import datetime

In [3]:
#Download Ericsson options data
raw = pd.read_csv('Ericsson.csv',sep=';', index_col=0, parse_dates=True)
raw.info()
raw

<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, ERICB0L30Y85 to ERICB3L150
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Exp.          177 non-null    object 
 1   Bid           154 non-null    float64
 2   Ask           166 non-null    float64
 3   Last          11 non-null     float64
 4   High          11 non-null     float64
 5   Low           11 non-null     float64
 6   Openinterest  80 non-null     object 
 7   Volume        11 non-null     float64
 8   Underlying    177 non-null    float64
 9   Strike        177 non-null    float64
dtypes: float64(8), object(2)
memory usage: 15.2+ KB


,Exp.,Bid,Ask,Last,High,Low,Openinterest,Volume,Underlying,Strike
Name,,,,,,,,,,
ERICB0L30Y85,2020-12-30,12.25,14.25,NaN,NaN,NaN,NaN,NaN,100.0,85.0
ERICB0L30Y87.50,2020-12-30,10.00,12.00,NaN,NaN,NaN,NaN,NaN,100.0,87.5
ERICB0L30Y90,2020-12-30,7.70,8.90,NaN,NaN,NaN,NaN,NaN,100.0,90.0
ERICB0L30Y92.50,2020-12-30,5.20,6.40,NaN,NaN,NaN,2,NaN,100.0,92.5
ERICB0L30Y95,2020-12-30,3.00,4.00,NaN,NaN,NaN,NaN,NaN,100.0,95.0
...,...,...,...,...,...,...,...,...,...,...
ERICB3L110,2023-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,110.0
ERICB3L120,2023-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,120.0
ERICB3L130,2023-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,130.0


In [8]:
#Rename Exp. as Maturity, short Date of expiration
raw.rename(columns={'Exp.':'Maturity'}, inplace=True)

In [9]:
#Create a copy so we do not have to reload the data set if we start over
data = raw.copy()

In [10]:
#Drop the column 'Underlying' since it only says how many stocks that an option contract is based on
data.drop(['Underlying'], axis = 1, inplace=True)

#Drop all rows where NA exists in the Date, Bid, Ask or Strike columns
data.dropna(subset=['Bid'], how='all', inplace=True)
data.dropna(subset=['Ask'], how='all', inplace=True)
data.dropna(subset=['Strike'], how='all', inplace=True)

#Drop any column that still has NAs
data.dropna(axis=1, how='any', inplace=True)

#Create a column with the market call option price from Bid and Ask column
data['Call'] = (data['Bid'] + data['Ask']) / 2

#Drop Bid and Ask column as we do not need them anymore
data.drop(['Bid'], axis = 1, inplace=True)
data.drop(['Ask'], axis = 1, inplace=True)

#Convert Maturity variable from string to datetime variable. 
data['Maturity']= pd.to_datetime(data['Maturity'], format='%Y-%m-%d') 

#Create a datetime variable, Date, 
data['Date'] = pd.to_datetime('2020/12/28', format='%Y-%m-%d') #2020/12/28 is the day data was downloaded
data

,Maturity,Strike,Call,Date
Name,,,,
ERICB0L30Y85,2020-12-30,85.0,13.25,2020-12-28
ERICB0L30Y87.50,2020-12-30,87.5,11.00,2020-12-28
ERICB0L30Y90,2020-12-30,90.0,8.30,2020-12-28
ERICB0L30Y92.50,2020-12-30,92.5,5.80,2020-12-28
ERICB0L30Y95,2020-12-30,95.0,3.50,2020-12-28
...,...,...,...,...
ERICB2L120,2022-12-16,120.0,7.30,2020-12-28
ERICB2L130,2022-12-16,130.0,5.25,2020-12-28
ERICB2L140,2022-12-16,140.0,3.65,2020-12-28


In [12]:
from ipynb.fs.full.Vasicek_calibration import Vasicek_calibration , r_list 
from ipynb.fs.full.Vasicek_zcb_valuation import zcb_price 

#Convert variables Maturity and Date to pandas datetime objects
data['Maturity']= pd.to_datetime(data['Maturity'], format='%Y-%m-%d') 
data['Date']= pd.to_datetime(data['Date'], format='%Y-%m-%d')

#Create a variable, T, that shows how much, as a fraction of a year, that is left until maturity
data['diff'] = data['Maturity'] - data['Date']                     #number of days left to maturity
data['T']=(data['diff'].astype('timedelta64[D]').astype(int))/365  

#Drop unnecessary varibles
data.drop(['diff'], axis = 1, inplace=True)                        
#Drop max.rows option so we can see full dataframe to decide where implied vol gives NaN values
pd.set_option('display.max_rows', None)                                          
data

,Maturity,Strike,Call,Date,T
Name,,,,,
ERICB0L30Y85,2020-12-30,85.0,13.250,2020-12-28,0.005479
ERICB0L30Y87.50,2020-12-30,87.5,11.000,2020-12-28,0.005479
ERICB0L30Y90,2020-12-30,90.0,8.300,2020-12-28,0.005479
ERICB0L30Y92.50,2020-12-30,92.5,5.800,2020-12-28,0.005479
ERICB0L30Y95,2020-12-30,95.0,3.500,2020-12-28,0.005479
ERICB0L30Y97.50,2020-12-30,97.5,1.350,2020-12-28,0.005479
ERICB0L30Y100,2020-12-30,100.0,0.405,2020-12-28,0.005479
ERICB1A08Y85,2021-01-08,85.0,13.500,2020-12-28,0.030137
ERICB1A08Y87.50,2021-01-08,87.5,11.000,2020-12-28,0.030137


In [16]:
#Keep only options with less than 1 year to maturity
data = data[data['T'] <= 1]

#Keep only options with at least one week to maturity
data = data[data['T'] >= 7/365]
options = data

In [14]:
#Initial short rate (Stibor 28.12.2020)
r0 = r_list[0]  

#Calibrate Short Rate Model to get calibrated estimates of Vasicek parameters used for zcb valuation
kappa_r, theta_r, sigma_r = Vasicek_calibration()                         

r = []
for row, option in options.iterrows():
    B0T = zcb_price([kappa_r, theta_r, sigma_r, r0, option['T']])                    
    r.append(-math.log(B0T) / option['T'])
options['r'] = r
options

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 153
         Function evaluations: 288


,Maturity,Strike,Call,Date,T,r
Name,,,,,,
ERICB1A08Y85,2021-01-08,85.0,13.500,2020-12-28,0.030137,-0.000491
ERICB1A08Y87.50,2021-01-08,87.5,11.000,2020-12-28,0.030137,-0.000491
ERICB1A08Y90,2021-01-08,90.0,8.500,2020-12-28,0.030137,-0.000491
ERICB1A08Y92.50,2021-01-08,92.5,6.000,2020-12-28,0.030137,-0.000491
ERICB1A08Y95,2021-01-08,95.0,3.900,2020-12-28,0.030137,-0.000491
ERICB1A08Y97.50,2021-01-08,97.5,2.150,2020-12-28,0.030137,-0.000491
ERICB1A08Y100,2021-01-08,100.0,0.900,2020-12-28,0.030137,-0.000491
ERICB1A08Y102.50,2021-01-08,102.5,0.405,2020-12-28,0.030137,-0.000491
ERICB1A08Y105,2021-01-08,105.0,0.405,2020-12-28,0.030137,-0.000491


In [15]:
data.to_csv(r'C:\Users\miche\Desktop\ERASMUS\PYTHON\Untitled Folder\Python_Seminar-Python-code\Python_Seminar-Python-code\new\option_data.csv', index = False)